In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import norm
from figaro.utils import rejection_sampler
from figaro.cosmology import CosmologicalParameters
from figaro import plot_settings

In [4]:
true_H0 = 70. # km/s/Mpc
Omega = CosmologicalParameters(true_H0/100., 0.315, 0.685, -1., 0., 0.)
norm_dist = norm()

In [5]:
# Mass distribution
from population_models.mass import plpeak

# dL distributions
# @njit
def dLsq(DL, DLmax = 5000):
    return 3*DL**2/DLmax**3

In [6]:
def selection_function(x):
    p = x/100
    p[x>100]=1
    return p


In [7]:
# Generate true detector frame mass samples
def generate_true_mz(n_draws_event):
    dL = rejection_sampler(n_draws_event, dLsq, [0, 5000])
    M  = rejection_sampler(n_draws_event, plpeak, [0,200])
    z  = Omega.Redshift(dL)
    return M * (1 + z)

In [12]:
# Generate detector frame mass posterior samples
def generate_mz_posterior_samples(Mz, sigma = 0.1):
    log_Mz_obs_mean = np.log(Mz) + sigma * norm_dist.rvs(len(Mz))
    return np.array([np.exp(np.random.normal(Mz_obs_mean_samp, sigma, 1000)) for Mz_obs_mean_samp in log_Mz_obs_mean])

In [13]:
Mz_max = 100
n_draws_event = 2000

Mz_sample = generate_true_mz(n_draws_event)
posterior = generate_mz_posterior_samples(Mz_sample[np.random.uniform(0,1,len(Mz_sample)) < Mz_sample/Mz_max])

/Users/thomas.ng/Documents/GitHub/pop_models_lvk/population_models/mass.py:29: RuntimeWarning: divide by zero encountered in power
  p = m**-alpha * (alpha-1.)/(mmin**(1.-alpha)-mmax**(1.-alpha))
/Users/thomas.ng/Documents/GitHub/pop_models_lvk/population_models/mass.py:10: RuntimeWarning: overflow encountered in exp
  p[idx]       = 1./(1.+np.exp(exponent))


In [14]:
np.savetxt('test_2/data/mz_true_samples.txt', Mz_sample)

In [15]:
posterior.shape

(516, 1000)

In [16]:
for i,p in enumerate(posterior):
    np.savetxt(f'test_2/data/mz_posterior_samples_{i+1}.txt', p)